In [2]:
import pandas as pd
from iexfinance.stocks import Stock
from datetime import datetime
import matplotlib.pyplot as plt
from iexfinance.stocks import get_historical_data
import os
import yaml
import sys
import datetime
import glob

# used to access data directory
sys.path.insert(0,'../')

In [3]:
import pandas as pd
table=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
df = table[0]
df.to_csv('S&P500-Info.csv')
df.to_csv("S&P500-Symbols.csv", columns=['Symbol'])

In [4]:
#  source ~/.ssh/iex_key.sh
# ENV var IEX_KEY
API_TOKEN=    credentials = yaml.load(open(os.path.expanduser(
        '~/.ssh/api_credentials.yml')))['iex']['api_key']

/Users/jonathanhilgart/Library/Caches/pypoetry/virtualenvs/daily-trading-nJ43NNNI-py3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  after removing the cwd from sys.path.


In [5]:
API_TOKEN

'pk_291949b3b45c490fa2a95869d87121c5'

In [6]:
sp = pd.read_csv('../data/S&P500-Symbols.csv', index_col=[0])


In [7]:
sp

,Symbol
0,MMM
1,ABT
2,ABBV
3,ABMD
4,ACN
...,...
500,YUM
501,ZBRA
502,ZBH
503,ZION


## test calls to IEX

In [8]:
# getCompanyInfo returns a dictionary with the company symbol as a key and the info as the value 
# call to iex finance api to return company info for a list of symbols
def getCompanyInfo(symbols):
    stock_batch = Stock(symbols, token=API_TOKEN)
    company_info = stock_batch.get_company()
    return company_info

In [9]:
sp_company_info = getCompanyInfo(sp["Symbol"][:5].tolist())


In [10]:
company_info_to_df = []
for company in sp_company_info:
    company_info_to_df.append(sp_company_info[company])

In [11]:
columns = ['symbol', 'companyName', 'exchange',
           'industry', 'website', 'CEO', 'sector']
company_info_to_df = pd.DataFrame(company_info_to_df, columns=columns )
company_info_to_df.head()

,symbol,companyName,exchange,industry,website,CEO,sector
0,MMM,3M Co.,New York Stock Exchange,Industrial Conglomerates,http://www.3m.com,Michael F. Roman,Producer Manufacturing
1,ABT,Abbott Laboratories,New York Stock Exchange,Medical Specialties,http://www.abbott.com,Robert B. Ford,Health Technology
2,ABBV,"AbbVie, Inc.",New York Stock Exchange,Pharmaceuticals: Major,http://www.abbvie.com,Richard A. Gonzalez,Health Technology
3,ABMD,"ABIOMED, Inc.",NASDAQ,Medical Specialties,http://www.abiomed.com,Michael R. Minogue,Health Technology
4,ACN,Accenture Plc,New York Stock Exchange,Information Technology Services,http://www.accenture.com,Julie Spellman Sweet,Technology Services


# Company Earnings

In [12]:
def getEarnings(symbol):
    stock_batch = Stock(symbol,
                        token=API_TOKEN)
    earnings = stock_batch.get_earnings(last=4)
    return earnings

single_stock_earnings = getEarnings(sp["Symbol"][0])

In [13]:
single_stock_earnings

[{'actualEPS': 2.16,
  'consensusEPS': 2.03,
  'announceTime': 'BTO',
  'numberOfEstimates': 17,
  'EPSSurpriseDollar': 0.13,
  'EPSReportDate': '2020-04-28',
  'fiscalPeriod': 'Q1 2020',
  'fiscalEndDate': '2020-03-31',
  'yearAgo': 2.23,
  'yearAgoChangePercent': -0.0314,
  'currency': 'USD'},
 {'actualEPS': 1.95,
  'consensusEPS': 2.11,
  'announceTime': 'BTO',
  'numberOfEstimates': 17,
  'EPSSurpriseDollar': -0.16,
  'EPSReportDate': '2020-01-28',
  'fiscalPeriod': 'Q4 2019',
  'fiscalEndDate': '2019-12-31',
  'yearAgo': 2.31,
  'yearAgoChangePercent': -0.1558,
  'currency': 'USD'},
 {'actualEPS': 2.58,
  'consensusEPS': 2.49,
  'announceTime': 'BTO',
  'numberOfEstimates': 16,
  'EPSSurpriseDollar': 0.09,
  'EPSReportDate': '2019-10-24',
  'fiscalPeriod': 'Q3 2019',
  'fiscalEndDate': '2019-09-30',
  'yearAgo': 2.58,
  'yearAgoChangePercent': 0,
  'currency': 'USD'},
 {'actualEPS': 2.2,
  'consensusEPS': 2.05,
  'announceTime': 'BTO',
  'numberOfEstimates': 16,
  'EPSSurpriseDoll

## Historical Prices

In [14]:
def getHistoricalPrices(stock,start_date, end_date):
    return get_historical_data(stock, start_date, end_date, 
                               output_format='pandas', 
                               token=API_TOKEN)
 
# start = datetime.datetime(2020, 4, 17)
# end = datetime.datetime.now() 
# single_stock_history = getHistoricalPrices('rost', start, end)

In [15]:
single_stock_history

NameError: name 'single_stock_history' is not defined

In [16]:
sp["Symbol"][0]

'MMM'

In [17]:
getHistoricalPrices('dia', start, end)

NameError: name 'start' is not defined

In [18]:
single_stock_history

NameError: name 'single_stock_history' is not defined

In [19]:
single_stock_history['close'].plot(label="3M Close")

NameError: name 'single_stock_history' is not defined

# Get list of historical prices for the first 100 stocks


In [20]:
sp[:100]

,Symbol
0,MMM
1,ABT
2,ABBV
3,ABMD
4,ACN
...,...
95,CE
96,CNC
97,CNP
98,CTL


In [21]:
start = datetime(2016, 1, 1)
end = datetime(2020, 7, 30)

TypeError: 'module' object is not callable

In [22]:
dict_of_stock_to_historical_price_df = {}

In [23]:

start.year, start.month, start.day

NameError: name 'start' is not defined

In [24]:
for stock in sp.Symbol[:5]:
    print(f"Downloading data for {stock}")
    stock_history = getHistoricalPrices(f"{stock}", start, end)
    dict_of_stock_to_historical_price_df[stock] = stock_history

NameError: name 'start' is not defined

In [25]:
update_s_and_p_dataset(20)

NameError: name 'update_s_and_p_dataset' is not defined

In [58]:



def update_s_and_p_dataset(stock_list=None, file_path=None, n_stock_to_download=100, default_start =datetime.datetime(2017, 1, 1), end = datetime.datetime.now() ):
    """
    Run through the S and P data and update

    Minimize API calls by only pulling new data
    """

    for stock in stock_list[:n_stock_to_download]:
        stock = stock.lower()
        print(f"downloading data for {stock}")
        we_have_data = False
        for file in glob.glob(f'{file_path}/*.csv'):
            # check every file to see if we have data on this stock

            if f"{stock}" in file: # we have a data file for this stock
                we_have_data = True
                
        if we_have_data:
            print(f'found existing stock for {stock}')
            # open file
         
            for file in glob.glob(f'{file_path}/*.csv'):
                
                if f"{stock}" in file:
                    print('file', f"{file}")
                    stock_df = pd.read_csv(f"{file}").set_index('date')
                    
                    
                    most_recent_date = stock_df.index.max().split(' ')[0]
                    oldest_date= stock_df.index.min().split(' ')[0]

                    
                    stock_df.index = pd.to_datetime(stock_df.index)

                    most_recent_year, most_recent_month, most_recent_day = most_recent_date.split('-')
                    oldest_year, oldest_month, oldest_day = oldest_date.split('-')

                    start = datetime.datetime(int(most_recent_year), int(most_recent_month), int(most_recent_day) + 1)
                    stock_history = getHistoricalPrices(f"{stock}", start, end)
                    
                    stock_history.index = pd.to_datetime(stock_history.index)

                    #combine new and old stocks
                    stock_history = pd.concat((stock_history, stock_df))

                    # delete the old file
                    os.remove(f"{file}")

        else: # we don't have this stocks data
            stock_history = getHistoricalPrices(f"{stock}", default_start, end)
            oldest_year=default_start.year
            oldest_month=default_start.month
            oldest_day=default_start.day
                
        now = stock_history.index.max()
        now_year = now.year
        now_month = now.month
        now_day = now.day
        # save the file
        print('file =', f'{file_path}/{stock}_{oldest_year}-{oldest_month}-{oldest_day}_to_{now_year}-{now_month}-{now_day}.csv')

        stock_history.to_csv(f'{file_path}/{stock}_{oldest_year}-{oldest_month}-{oldest_day}_to_{now_year}-{now_month}-{now_day}.csv')





In [47]:
# our test stocks
list_of_20_stocks = [ 'adbe', 'are', 'atvi', 'bac', 'ce', 'cern', 'efx', 'emr', 'eqix', 'lb', 'ni', 'peg', 'pg', 'ph', 'pnr', 'pvh', 'rop', 'rost', 'so', 'ual', 'vno']

In [48]:
# reits are stocks
reit_stock_list = [
'vno',
'SUI',
'DLR',
'NRZ',
'REG',
'FRT',
'CPT',
'NLY',
'PLD',
'HASI',
'WELL',
'DRE',
'SRG',
'PEI',
'IRM',
'LAMR',
'NRZ',
'AGNC',
'HIW',
'eqix',
'are'
]

In [49]:
bank_stock_list = ['C',
'BKU',
'CMA',
'STT',
'FITB',
'FTC',
'WFC',
'MTB',
'USB',
'BAC',
'KEY',
'JPM',
'PNC',
'BKU',
'NTRS',
'GBCI',
'PACW',
'WAL',
'FSBW',
'HTBK',
'BCML',
'FHB',
'HAFC',
'CATY',
'BOH',
'COLB',
'CVCY',
'OVLY',
'HOPE',
'CWBC',
'UMPQ',
'UBFO',
'BSRR',
'CVBF',
'TCBK',
'RBB',
'PUB',
'CPG',
'EWBC',
'BMRC',
'PFBC',
'ZION',
'WABC',
'AMRB',
'BOCH',
'FRC',
'SIVB',
'CCB']

In [50]:
# retail_stocks = ['m', 'kss','dds','dbi','anf','aeo','zumz','jwn','gps','lb','tpr','cpri','fl','ctrn','goos','spwh','plce','bke','urbn','gco','scvl','cato','boot',
#                 'levi','ftch','tjx','rost']

In [51]:
list_of_etfs = ['dia', 'spy', 'iwm', 'qqq', 'pej', 'pbj', 'vti',  'vfh', 'ibuy', 'xrt',  'inn', 'bjk']

In [63]:
file_path_stocks = '../data/historical_stock_data'
file_path_etf = '../data/historical_etf_data'
update_s_and_p_dataset(file_path=file_path_stocks, stock_list=reit_stock_list) # are reits in stokcs?

downloading data for vno
file = ../data/historical_stock_data/vno_2017-1-1_to_2020-5-15.csv
downloading data for sui
file = ../data/historical_stock_data/sui_2017-1-1_to_2020-5-15.csv
downloading data for dlr
file = ../data/historical_stock_data/dlr_2017-1-1_to_2020-5-15.csv
downloading data for nrz
file = ../data/historical_stock_data/nrz_2017-1-1_to_2020-5-15.csv
downloading data for reg
file = ../data/historical_stock_data/reg_2017-1-1_to_2020-5-15.csv
downloading data for frt
file = ../data/historical_stock_data/frt_2017-1-1_to_2020-5-15.csv
downloading data for cpt
file = ../data/historical_stock_data/cpt_2017-1-1_to_2020-5-15.csv
downloading data for nly
file = ../data/historical_stock_data/nly_2017-1-1_to_2020-5-15.csv
downloading data for pld
file = ../data/historical_stock_data/pld_2017-1-1_to_2020-5-15.csv
downloading data for hasi
file = ../data/historical_stock_data/hasi_2017-1-1_to_2020-5-15.csv
downloading data for well
file = ../data/historical_stock_data/well_2017-1-1_t

In [44]:
dict_of_stock_to_historical_price_df['MMM'].index.max() # index is dat as returned from the API

Timestamp('2020-04-17 00:00:00')

In [70]:
mmm_df = pd.read_csv('../data/MMM_2016-01-04_to_2020-04-17.csv')

In [72]:
mmm_df.date.max()

'2020-04-17'

In [68]:
dict_of_stock_to_historical_price_df['MMM'].to_csv('../data/MMM_2016-01-04_to_2020-04-17.csv')

In [61]:
import os

directory = '../data/historical_etf_data'

[os.rename(os.path.join(directory, f), os.path.join(directory, f).lower()) for f in os.listdir(directory)]

[None, None, None, None, None, None, None, None, None, None, None, None, None]